In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Point Managers
todo

## HasPts Objects in Decod.es

### HasPts Members and Methods

## PLine Objects in Decod.es

# Accessing and Manipulating HasPts Objects
todo

# PGon and RGon Objects in Decod.es
todo

### PGons in Decod.es

### RGons in Decod.es